# Phase-field simulation of spinodal decomposition

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MatSciEd/Thermodynamics/blob/main/3.%20Spinodal%20Decomposition/3.%20Spinodal%20Decomposition.ipynb)

This notebook is based on the pystencils tutorial at https://pycodegen.pages.i10git.cs.fau.de/pystencils/notebooks/05_tutorial_phasefield_spinodal_decomposition.html.

This simulation implement a simple phase field models using finite differences using pystencil.

Phase field models evolve the mole fraction and chemical potentials of a material forward to describe how a system approaches thermodynamic equilibrium.

The movie visualization requires that you install the FFMPEG tool on your computer, which is available at https://www.ffmpeg.org/download.html.

In [2]:
# Import NumPy numerical package
import numpy as np

# Install pystencils package using pip in the current Jupyter kernel
# To use pystencils, restart the kernel
import sys
!{sys.executable} -m pip install "pystencils[interactive]"

# Import pystencils package
from pystencils.session import *

# Import matplotlib and set defaults
import matplotlib.animation as animation
plt.rcParams['figure.figsize'] = [16, 16]
plt.rcParams.update({'font.size': 18})

ModuleNotFoundError: No module named 'pystencils'

First we create a DataHandling instance, that manages the numpy arrays and their corresponding symbolic *sympy* fields. We create two arrays of size $256 \times 256$, one for the concentration $c$ and one for the chemical potential $\mu$, on a 2D periodic simulation box.

In [ ]:
dh = ps.create_data_handling(domain_size=(256, 256), periodicity=True)
μ_field = dh.add_array('mu', latex_name='μ')
c_field = dh.add_array('c')

Next we calculate the Gibbs free energy density, consisting of a bulk and an interface component. The bulk free energy is minimal in regions where only either phase 0 or phase 1 is present. Areas of mixture are penalized.

The interfacial free energy penalized regions where the gradient of the phase field is large, i.e. it tends to smear out the interface. The strength of these counteracting contributions is balanced by the parameters 
$A$ or the bulk- and $\kappa$ for the interface part. The ratio of these parameters determines the interface width.

In [ ]:
κ, A = sp.symbols("κ A")

c = c_field.center
μ = μ_field.center

def f(c):
    return A * c**2 * (1-c)**2

bulk_free_energy_density = f(c)
grad_sq = sum(ps.fd.diff(c, i)**2 for i in range(dh.dim))
interfacial_free_energy_density = κ/2 * grad_sq

free_energy_density = bulk_free_energy_density + interfacial_free_energy_density
free_energy_density

The index of the concentration $c$ enumerates the discretized set of boxes for concentration field described by an array. This becomes important when we apply a finite difference discretization on the equation.

We use the bulk free energy $c^2 (1-c)^2$ as it is the simplest polynomial with minima at $c=0$ and $c=1$.

In [ ]:
# Plot the model for the bulk Gibbs free energy
plt.figure(figsize=(8,6))
plt.sympy_function(bulk_free_energy_density.subs(A, 1), (-0.2, 1.2))
plt.xlabel("c")
plt.title("Bulk Gibbs free energy");

To minimize the total free energy we use the diffusion or Cahn Hilliard equation
$$
\frac{\partial c}{\partial t} = \nabla \cdot \left( M \nabla \frac{\delta G}{\delta c} \right),
$$
where the functional derivative $\delta G / \delta c$ is the chemical potential $\mu$. The functional derivative $\delta G /  \delta c$ is computed as
$$
\frac{\delta G}{\delta c} = \frac{\partial G}{\partial c} - \nabla \cdot \frac{\partial G}{\partial \nabla c}.
$$
That means we treat the gradient of the concentration $\nabla c$ like a normal variable when calculating derivatives. The Python package pystencils offers a function to do just that.

In [ ]:
ps.fd.functional_derivative(free_energy_density, c)

In this case we could quite simply do this derivative by hand but for more complex phase field models this step is quite tedious. If we discretize this term using finite differences, we have a computation rule how to compute the chemical potential $\mu$ from the free energy.

In [ ]:
discretize = ps.fd.Discretization2ndOrder(dx=1, dt=0.01)

μ_update_eq = ps.fd.functional_derivative(free_energy_density, c)
μ_update_eq = ps.fd.expand_diff_linear(μ_update_eq, constants=[κ])  # pull constant κ in front of the derivatives
μ_update_eq_discretized = discretize(μ_update_eq)
μ_update_eq_discretized

Pystencils computes the finite difference approximation for us. This was only possible since all symbols occuring inside derivatives are pystencils field variables, so that neighboring values can be accessed. Next we bake this formula into a kernel that writes the chemical potential to a field. Therefor we first insert the $\kappa$ and $A$ parameters, build an assignment out of it, and compile the kernel

In [ ]:
μ_kernel = ps.create_kernel([ps.Assignment(μ_field.center,
                                           μ_update_eq_discretized.subs(A, 1).subs(κ, 0.5))]
                           ).compile()

Next, we formulate the Cahn-Hilliard equation itself, which is just a diffusion equation.

In [ ]:
M = sp.Symbol("M")
cahn_hilliard = ps.fd.transient(c) - ps.fd.diffusion(μ, M)
cahn_hilliard

It can be given right away to the `discretize` function, that by default uses a simple explicit Euler scheme for temporal, and second order finite differences for spatial discretization. It returns the update rule for the concentration field.

In [ ]:
c_update = discretize(cahn_hilliard)
c_update

Pystencil uses kernels and we can build a kernel from this update rule:

In [ ]:
c_kernel = ps.create_kernel([ps.Assignment(c_field.center,
                                           c_update.subs(M, 1))]
                           ).compile()

Before we run the simulation, the domain has to be initialized. To access a numpy array inside a data handling we have to iterate over the data handling. This somewhat complicated way is necessary to be able to switch to distributed memory parallel simulations without having to alter the code. Basically this loops says “iterate over the portion of the domain that belongs to my process”, which in our serial case here is just the full domain.

We initialize everything with $c = 0.4$ and add some random noise on top of it.

In [ ]:
def init(value=0.4, noise=0.02):
    for b in dh.iterate():
        b['c'].fill(value)
        np.add(b['c'], noise*np.random.rand(*b['c'].shape), out=b['c'])

The time loop of the simulation is straightforward. We call the kernels to update the chemical potential and the concentration in alternating fashion. In between we have to do synchronization steps for the fields that take care of the periodic boundary condition, and in the parallel case of the communciation between processes.

In [ ]:
def timeloop(steps=20):
    c_sync = dh.synchronization_function(['c'])
    μ_sync = dh.synchronization_function(['mu'])
    c1 = dh.gather_array('c')
    for t in range(steps):
        c_sync()
        dh.run_kernel(μ_kernel)
        μ_sync()
        dh.run_kernel(c_kernel)
    return c1
init()

Now we can run the simulation and see how the the spinodal transformation leads to an increase of concentration fluctuations and to phases separation.

In [ ]:
ani = ps.plot.scalar_field_animation(timeloop, rescale=True, interval=1, frames=1000)
FFwriter = animation.FFMpegWriter()
#ani.save("animation.mp4", fps = 50, extra_args = ['-vcodec', 'libx264'])

result = ps.jupyter.display_as_html_video(ani)
result